In [0]:
df_amsterdam  = spark.read.format("delta").table("bronze.amsterdam")
df_athens     = spark.read.format("delta").table("bronze.athens")
df_barcelona  = spark.read.format("delta").table("bronze.barcelona")
df_berlin     = spark.read.format("delta").table("bronze.berlin")
df_lisbon     = spark.read.format("delta").table("bronze.lisbon")
df_london     = spark.read.format("delta").table("bronze.london")
df_paris      = spark.read.format("delta").table("bronze.paris")
df_rome       = spark.read.format("delta").table("bronze.rome")


In [0]:
from pyspark.sql.functions import current_timestamp, lit

df_amsterdam  = df_amsterdam.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("amsterdam"))
df_athens     = df_athens.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("athens"))
df_barcelona  = df_barcelona.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("barcelona"))
df_berlin     = df_berlin.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("berlin"))
df_lisbon     = df_lisbon.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("lisbon"))
df_london     = df_london.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("london"))
df_paris      = df_paris.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("paris"))
df_rome       = df_rome.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_tabela", lit("rome"))


In [0]:
from pyspark.sql import functions as F

# ---------- Helpers ----------
def _apply_name_rules_ptbr(colname: str) -> str:
    """
    Aplica regras de renomeação para nomes claros em PT-BR.
    Substituições específicas para o seu dataset.
    """
    col_map = {
        "_C0": "CODIGO",
        "REALSUM": "PRECO_TOTAL_DIARIO",
        "ROOM_TYPE": "TIPO_QUARTO",
        "ROOM_SHARED": "QUARTO_COMPARTILHADO",
        "ROOM_PRIVATE": "QUARTO_PRIVADO",
        "PERSON_CAPACITY": "CAPACIDADE_MAXIMA",
        "HOST_IS_SUPERHOST": "ANFITRIAO_SUPERHOST",
        "MULTI": "IMOVEIS_ANFITRIAO",
        "BIZ": "HOSPEDAGEM_CORPORATIVA",
        "CLEANLINESS_RATING": "NOTA_LIMPEZA",
        "GUEST_SATISFACTION_OVERALL": "NOTA_SATISFACAO_HOSPEDES",
        "BEDROOMS": "NUMERO_QUARTOS",
        "DIST": "DISTANCIA_CENTRO_KM",
        "METRO_DIST": "DISTANCIA_METRO_KM",
        "ATTR_INDEX": "INDICE_ATRACOES",
        "ATTR_INDEX_NORM": "INDICE_ATRACOES_NORMALIZADO",
        "REST_INDEX": "INDICE_RESTAURANTES",
        "REST_INDEX_NORM": "INDICE_RESTAURANTES_NORMALIZADO",
        "LNG": "LONGITUDE",
        "LAT": "LATITUDE",
    }
    return col_map.get(colname.upper(), colname.upper())



# ---------- Core ----------
def renomear_colunas_cidades(src_fqn: str, dest_fqn: str = None):
    """
    Lê uma tabela Delta do metastore (Bronze), aplica nomes claros em PT-BR,
    ajusta auditoria e salva como tabela Silver.
    - src_fqn: 'schema.tabela' de origem (ex.: 'bronze.rome')
    - dest_fqn: 'schema.tabela' de destino; se None, sobrescreve a própria origem
    """
    dest_fqn = dest_fqn or src_fqn

    # Lê como tabela managed
    df = spark.read.format("delta").table(src_fqn)

    # Renomeia todas as colunas de uma vez
    new_cols = [_apply_name_rules_ptbr(c) for c in df.columns]
    df = df.toDF(*new_cols)

    # Adiciona colunas de auditoria Silver
    df = (
        df
        .withColumn("DATA_HORA_BRONZE", F.lit(src_fqn))  # origem rastreável
        .withColumn("DATA_ARQUIVO_SILVER", F.current_timestamp())
    )

    # Salva como tabela managed Delta (sobrescrevendo destino)
    df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(dest_fqn)


    return dest_fqn


In [0]:
renomear_colunas_cidades("bronze.rome", "silver.rome")  
renomear_colunas_cidades("bronze.amsterdam", "silver.amsterdam")
renomear_colunas_cidades("bronze.athens", "silver.athens")
renomear_colunas_cidades("bronze.barcelona", "silver.barcelona")
renomear_colunas_cidades("bronze.berlin", "silver.berlin")
renomear_colunas_cidades("bronze.lisbon", "silver.lisbon")
renomear_colunas_cidades("bronze.paris", "silver.paris")
renomear_colunas_cidades("bronze.london", "silver.london")
